<style>
@font-face {font-family: "B Nazanin"; src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot"); src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot?#iefix") format("embedded-opentype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff2") format("woff2"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff") format("woff"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.ttf") format("truetype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.svg#B Nazanin") format("svg"); }
    </style>
<div dir="rtl" align="center" style ='font-family: "B Nazanin";'>
    <h1>
        تمرین سوم درس پردازش زبان‌های طبیعی
    </h1>
    <h3>
        گردآورندگان:<br/>
        ساحل مس‌فروش، سروش تابش، درنا دهقانی
    </h3>
    
</div>

<br/>
<br/>


<div dir="rtl"  style ='font-family: "B Nazanin";'>
    ما در این تمرین ترک "تکمیل مصراع دوم با رعایت وزن شعر" را انتخاب کردیم. در این تمرین قصد داریم به کمک مدل‌های زبانی مناسب، چند کلمه به عنوان یک مصرع از بیت را به عنوان ورودی بگیریم و چند کلمه به عنوان مصرع دوم این بیت را خروجی دهیم. در این تمرین ما از وبسایت گنجور برای جمع‌آوری داده کمک گرفتیم و از چندین قالب شعری استفاده کردیم، زیرا قافیه داشتن یا نداشتن به عنوان ورودی داده می‌شود. اگر نیاز به هم‌قافیه بودن دو مصراع بود، از قالب مثنوی نظیر شاهنماه‌ی فردوسی کمک می‌گیریم؛ و اگر نیاز به هم‌قافیه بودن دو مصراع نبود می‌توانیم از قالب قطعه استفاده کنیم. <br>
    برای ساخت چنین سیستمی از چند مدل زبانی استفاده می‌کنیم: مدل n-gram، مدل encoder-decoder با یک شبکه LSTM به عنوان encoder، و مدلی پیرو مکانیسم توجه.
</div>

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    <h3> 1. دوباره‌نویسی داده </h3>
    از آنجایی که در داده‌ی گرفته شده، ابتدا و انتهای ابیات و مصراع‌ها مشخص نبود، به کمک کد زیر این داده را بازنویسی کردیم. ابتدا و انتهای ابیات با __BOB__ و __EOB__ و ابتدا و انتهای مصراع‌ها با __BOM__ و __EOM__ مشخص گردید.
</div>

In [5]:
from transformers import (
    AutoTokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    AutoModelWithLMHead,
    BertModel
)
import codecs
import os
import glob
from __future__ import unicode_literals
from hazm import *
import os
from transformers import pipeline, AutoTokenizer, GPT2LMHeadModel
import torch
print(torch.cuda.is_available())

True


In [3]:
! pip install pandas
! pip install hazm
! pip install transformers
! pip install torch

In [7]:
class Cleaner:
    def __init__(self):
        self.normalizer = Normalizer()
        # self.stop_words = [self.normalizer.normalize(x.strip()) for x in codecs.open('data/stopwords.txt','r','utf-8').readlines()]
        self.file_range = None

    def add_statement_mesras(self, file_in= "./data/ferdousi_norm.txt", file_out='./data/f_beyt.txt'):
        start_mesra = '[BOM]'
        end_mesra = ''
        start_beyt = ''
        end_beyt = '[EOS]' + os.linesep
        beyt_file = ''
        with open(file_in, 'r', encoding="utf-8") as fp:
            lines = fp.readlines()
            for i in range(0, len(lines) - 1, 2):
                mesra1 = start_mesra + lines[i].strip() + end_mesra
                mesra2 = start_mesra + lines[i + 1].strip() + end_mesra
                b = start_beyt + mesra1.strip() + ' ' + mesra2.strip() + end_beyt
                beyt_file += b
                
        with open(file_out, 'w', encoding="utf-8") as fp:
            fp.write(beyt_file)

    def read_poems(self, file_patterns: str, file_range: tuple, normalize=False):  #NOT USED
        file_range = range(*file_range) if file_range is not None else None 
        file_names = glob.glob(file_patterns)
        mesra_collection = []
        for file_name in file_names:
            if normalize:
                mesra_collection += [self.normalizer.normalize(x) for x in codecs.open(file_name,'rU','utf-8').readlines()]
            else:
                mesra_collection += codecs.open(file_name,'rU','utf-8').readlines()[2:]
        return mesra_collection
    

    def read_documents(self): #NOT USED
        poems = self.read_poems(self.file_pattern, self.file_range)
        cleaned = self.clean_data(poems)
        return ' \n '.join(cleaned)
    

In [8]:
class MesraGenerator:
    def __init__(self, train_path, test_path):
        self.base_path = "./data"
        self.model = None
        self.tokenizer = None
        self.generator = None
        self.train_path = os.path.join(self.base_path, train_path)
        self.test_path = os.path.join(self.base_path, test_path)
        # self.cleaner = Cleaner()

    def read_data(self, tokenizer, train_path=None, test_path=None):
        path = train_path if train_path is not None else self.train_path
        test_path = test_path if test_path is not None else self.test_path
        train_dataset = TextDataset(
            tokenizer=tokenizer, file_path=path, block_size=128)
        
        test_dataset = TextDataset(
            tokenizer=tokenizer, file_path=test_path, block_size=128)
        
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=tokenizer,
            mlm=False,
        )
        # train_texts, val_texts = train_test_split(train_dataset)
        return train_dataset, test_dataset, data_collator

    def read_model(self, model_type='bolbolzaban/gpt2-persian'):
        self.model = GPT2LMHeadModel.from_pretrained(model_type)
        self.tokenizer = AutoTokenizer.from_pretrained('bolbolzaban/gpt2-persian')
        return self.model, self.tokenizer

    def freeze_lower_layers(self):
        for param in self.model.base_model.parameters():
            param.requires_grad = False

        for param in (
            self.model.base_model.h[23].parameters() or self.model.base_model.h[22].parameters()
        ):
            param.requires_grad = True
            
    def fine_tune_model(self, model, tokenizer, train_texts, val_texts, data_collator):
        training_args = TrainingArguments(
        output_dir="./model",
        overwrite_output_dir=True,
        num_train_epochs=5,
        # Set the batch size to a maximum value that could fit into GPU memory,
        # for example 12 is the largest batch size that could work on a 6gb GPU when training the last to layers
        per_device_train_batch_size=12,
        per_device_eval_batch_size=12,
        eval_steps=1000,
        save_steps=1000,
        warmup_steps=500)

        trainer = Trainer(
            model=model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=train_texts,
            eval_dataset=val_texts,
        )
        trainer.train()
        return trainer
        
    def init_generator(self):      
        model, tokenizer = self.read_model()
        self.freeze_lower_layers()
        train_texts, val_texts, data_collator = self.read_data(tokenizer)
        trainer = self.fine_tune_model(model, tokenizer, train_texts, val_texts, data_collator)
        generator = pipeline('text-generation', trainer.model, tokenizer=tokenizer, config={'max_length':256}, device=0)
        self.generator = generator
        return generator

    def save_model(self, dir=None):
        self.trainer.save_model(output_dir=dir)

In [5]:
cleaner = Cleaner()
cleaner.add_statement_mesras()

49606


In [ ]:
train_path = "f_beyt.txt"
test_path = "test.txt"
mesra_model = MesraGenerator(train_path, test_path)
generator = mesra_model.init_generator()
# train_texts, val_texts, data_collator = generator.read_data()
generator('[BOM]به گرد اندر آرد بهنگام کار')